In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Завдання 1:

Завантаження даних та їх попередній аналіз

In [2]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

df = pd.concat([X, y], axis=1)

1. Перші 5 рядків

In [3]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


2. Інформація про стовпці та типи

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

Перевірка пропусків


In [5]:
df.isnull().sum()

mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
target                     0
dtype: int64

Розмір даних

In [6]:
print(f"Рядків: {df.shape[0]}, Ознак: {df.shape[1]-1}")

Рядків: 569, Ознак: 30


Побудова і налаштування моделей

In [19]:
# розділення на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# масштабування
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Побудова моделей та GridSearch

# словник моделей та їх параметрів для перебору
model_params = {
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.1, 1, 10],
            'solver': ['liblinear', 'lbfgs']
        }
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(random_state=17),
        'params': {
            'max_depth': [None, 5, 10, 20],
            'min_samples_split': [2, 5, 10],
            'criterion': ['gini', 'entropy']
        }
    },
    'RandomForest': {
        'model': RandomForestClassifier(random_state=17),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]
        }
    }
}

best_models = {}
best_score = 0
best_model_name = ""

print("\nПошук найкращих параметрів (GridSearch):")

for name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, scoring='accuracy', n_jobs=-1)
    clf.fit(X_train_scaled, y_train)

    print(f"\nModel: {name}")
    print(f"Best Params: {clf.best_params_}")
    print(f"Best CV Score: {clf.best_score_:.4f}")

    best_models[name] = clf.best_estimator_

    # оцінка на тестових даних
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)

    print(f"Test Accuracy: {acc:.4f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Report:\n", classification_report(y_test, y_pred))

    if acc > best_score:
        best_score = acc
        best_model_name = name


Пошук найкращих параметрів (GridSearch):

Model: LogisticRegression
Best Params: {'C': 0.1, 'solver': 'lbfgs'}
Best CV Score: 0.7992
Test Accuracy: 0.7989
Confusion Matrix:
 [[97 13]
 [23 46]]
Report:
               precision    recall  f1-score   support

           0       0.81      0.88      0.84       110
           1       0.78      0.67      0.72        69

    accuracy                           0.80       179
   macro avg       0.79      0.77      0.78       179
weighted avg       0.80      0.80      0.80       179


Model: DecisionTree
Best Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10}
Best CV Score: 0.8189
Test Accuracy: 0.7933
Confusion Matrix:
 [[95 15]
 [22 47]]
Report:
               precision    recall  f1-score   support

           0       0.81      0.86      0.84       110
           1       0.76      0.68      0.72        69

    accuracy                           0.79       179
   macro avg       0.79      0.77      0.78       179
weighted 

Прогнозування і висновки

In [8]:
print(f"НАЙКРАЩА МОДЕЛЬ: {best_model_name} з точністю {best_score:.4f}")

final_model = best_models[best_model_name]
final_predictions = final_model.predict(X_test_scaled)

comparison_df = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': final_predictions
}).head(10)

print("\nРезультати прогнозування (перші 10):")
print(comparison_df)

НАЙКРАЩА МОДЕЛЬ: LogisticRegression з точністю 0.9737

Результати прогнозування (перші 10):
   Actual  Predicted
0       0          0
1       1          1
2       0          0
3       1          1
4       0          0
5       1          1
6       1          1
7       0          0
8       0          0
9       0          0


Завдання 2:

Завантаження набору даних:

In [20]:
import kagglehub
path = kagglehub.dataset_download("yasserh/titanic-dataset")

In [10]:
import os
for file in os.listdir(path):
    if file.endswith(".csv"):
        csv_path = os.path.join(path, file)
        break

df2 = pd.read_csv(csv_path)

Попередній аналіз даних

In [11]:
print(df2.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [12]:
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [13]:
print(df2.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


Очистка та підготовка даних

In [14]:
df_clean = df2.copy()

# Обробка пропусків
# Age заповнюємо медіаною
df_clean['Age'] = df_clean['Age'].fillna(df_clean['Age'].median())

# Embarked заповнюємо модою
if df_clean['Embarked'].isnull().sum() > 0:
    df_clean['Embarked'] = df_clean['Embarked'].fillna(df_clean['Embarked'].mode()[0])

# Видалення зайвих стовпців
# Cabin - забагато пропусків
# Інші - текстові ідентифікатори, що не впливають на виживання напряму
cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df_clean = df_clean.drop(columns=cols_to_drop)

# Кодування категорій (Sex, Embarked) у числа
le = LabelEncoder()
df_clean['Sex'] = le.fit_transform(df_clean['Sex']) # male/female -> 1/0
df_clean['Embarked'] = le.fit_transform(df_clean['Embarked'].astype(str))

print("Дані після очистки та кодування")
print(df_clean.info())
print(df_clean.head())

Дані після очистки та кодування
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 55.8 KB
None
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0         0       3    1  22.0      1      0   7.2500         2
1         1       1    0  38.0      1      0  71.2833         0
2         1       3    0  26.0      0      0   7.9250         2
3         1       1    0  35.0      1      0  53.1000         2
4         0       3    1  35.0      0      0   8.0500         2


ПІДГОТОВКА ДО НАВЧАННЯ

In [15]:
X = df_clean.drop('Survived', axis=1)
y = df_clean['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

ПОБУДОВА 3 МОДЕЛЕЙ ТА GRIDSEARCH

In [16]:
model_params = {
    'Random Forest': {
        'model': RandomForestClassifier(random_state=17),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [5, 10, None],
            'min_samples_leaf': [1, 2]
        }
    },
    'SVC (Support Vector Machine)': {
        'model': SVC(random_state=17),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['rbf', 'linear'],
            'gamma': ['scale', 'auto']
        }
    },
    'KNN (K-Nearest Neighbors)': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9, 11],
            'weights': ['uniform', 'distance']
        }
    }
}

best_model_name = ""
best_accuracy = 0
best_trained_model = None

print("\nПочаток навчання та підбору параметрів моделей:")

for name, config in model_params.items():
    gs = GridSearchCV(config['model'], config['params'], cv=5, scoring='accuracy', n_jobs=-1)
    gs.fit(X_train_sc, y_train)

    # передбачення на тесті
    y_pred = gs.predict(X_test_sc)
    acc = accuracy_score(y_test, y_pred)

    print(f"\nМодель: {name}")
    print(f"  Найкращі параметри: {gs.best_params_}")
    print(f"  Точність (Accuracy) на тесті: {acc:.4f}")

    # збереження найкращої моделі
    if acc > best_accuracy:
        best_accuracy = acc
        best_model_name = name
        best_trained_model = gs.best_estimator_


Початок навчання та підбору параметрів моделей:

Модель: Random Forest
  Найкращі параметри: {'max_depth': 10, 'min_samples_leaf': 2, 'n_estimators': 100}
  Точність (Accuracy) на тесті: 0.8045

Модель: SVC (Support Vector Machine)
  Найкращі параметри: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
  Точність (Accuracy) на тесті: 0.7877

Модель: KNN (K-Nearest Neighbors)
  Найкращі параметри: {'n_neighbors': 9, 'weights': 'distance'}
  Точність (Accuracy) на тесті: 0.7654


ВИСНОВКИ ТА ПРОГНОЗ

In [17]:
print(f"Найкраща модель: {best_model_name} з точністю {best_accuracy:.4f}")

# Детальний звіт для переможця
final_pred = best_trained_model.predict(X_test_sc)
print("\n--- Classification Report (для найкращої моделі) ---")
print(classification_report(y_test, final_pred))

print("--- Confusion Matrix ---")
print(confusion_matrix(y_test, final_pred))

Найкраща модель: Random Forest з точністю 0.8045

--- Classification Report (для найкращої моделі) ---
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       106
           1       0.79      0.71      0.75        73

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179

--- Confusion Matrix ---
[[92 14]
 [21 52]]


прогноз для 10 випадкових пасажирів

In [18]:
random_indices = np.random.choice(len(X_test), 10, replace=False)

X_sample_sc = X_test_sc[random_indices]
X_sample_orig = X_test.iloc[random_indices].copy()
y_sample_true = y_test.iloc[random_indices].values

#  прогноз
y_sample_pred = best_trained_model.predict(X_sample_sc)

results_df = X_sample_orig[['Pclass', 'Sex', 'Age', 'Fare']].copy()

sex_map = {0: 'female', 1: 'male'}
results_df['Sex'] = results_df['Sex'].map(sex_map)

results_df['Real Survived'] = y_sample_true
results_df['Pred Survived'] = y_sample_pred
results_df['Correct?'] = results_df['Real Survived'] == results_df['Pred Survived']

print("\nПрогноз для 10 випадкових пасажирів")
print(results_df.to_string())


Прогноз для 10 випадкових пасажирів
     Pclass     Sex   Age      Fare  Real Survived  Pred Survived  Correct?
556       1  female  48.0   39.6000              1              1      True
336       1    male  29.0   66.6000              0              0      True
641       1  female  24.0   69.3000              1              1      True
776       3    male  28.0    7.7500              0              0      True
633       1    male  28.0    0.0000              0              0      True
105       3    male  28.0    7.8958              0              0      True
58        2  female   5.0   27.7500              1              1      True
759       1  female  33.0   86.5000              1              1      True
265       2    male  36.0   10.5000              0              0      True
498       1  female  25.0  151.5500              0              1     False


Висновки: <br>
У ході виконання завдань було проведено аналіз та побудова моделей машинного навчання на двох різних наборах даних: Breast Cancer та Titanic Dataset. Для обох наборів даних було виконано попередню обробку, включаючи очищення даних, обробку пропусків та кодування категоріальних змінних. Було використано метод GridSearchCV для підбору найкращих гіперпараметрів моделей, що дозволило підвищити їх точність. Найкращі моделі були визначені на основі точності на тестових даних, і для кожної моделі було надано детальний звіт про продуктивність.